In [20]:
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from datetime import datetime

# Path to the export.xml file
xml_file = 'apple_health_export/export.xml'


### Understanding the XML Structure 📄

The raw XML data exported from Apple Health includes **workout details** with nested tags that provide additional metrics. Here's a breakdown of the structure:

- **Workout Attributes**:
   - `workoutActivityType`: Specifies the type of activity (e.g., Running).
   - `duration`: Duration of the workout.
   - `startDate` and `endDate`: Workout start and end times.

- **Metadata Entries** (`MetadataEntry`):
   - Contains additional details like `HKAverageMETs` (average METs) and `HKIndoorWorkout` (indoor/outdoor flag).

- **Workout Events** (`WorkoutEvent`):
   - Tracks workout segments, pauses, and resumes with:
     - `type`: Type of event (e.g., Segment, Pause, Motion Resumed).
     - `date`: Timestamp of the event.
     - `duration`: Duration of the segment, if applicable.

- **Workout Statistics** (`WorkoutStatistics`):
   - Includes cumulative metrics such as:
     - `HKQuantityTypeIdentifierActiveEnergyBurned`: Calories burned.
     - `HKQuantityTypeIdentifierDistanceWalkingRunning`: Distance covered.

- **Workout Route**:
   - Provides a path to the GPX route file for GPS data.

Below is the Python code to **parse** this structure and extract relevant data into clean, analyzable tables.


### STRUCTURE OF TREE DATA
```xml
<Workout workoutActivityType="HKWorkoutActivityTypeRunning" duration="17.12425581614177" durationUnit="min" sourceName="Jason’s Apple Watch" sourceVersion="6.1.1" device="&lt;&lt;HKDevice: 0x3023054f0&gt;, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch2,4, software:6.1.1, creation date:2019-12-25 05:23:41 +0000&gt;" creationDate="2020-01-07 12:10:31 -0600" startDate="2020-01-07 11:52:25 -0600" endDate="2020-01-07 12:10:21 -0600">
  <MetadataEntry key="HKAverageMETs" value="11.1312 kcal/hr·kg"/>
  <MetadataEntry key="HKTimeZone" value="America/Chicago"/>
  <MetadataEntry key="HKIndoorWorkout" value="0"/>
  <WorkoutEvent type="HKWorkoutEventTypeSegment" date="2020-01-07 11:52:25 -0600" duration="6.331976975997289" durationUnit="min"/>
  <WorkoutEvent type="HKWorkoutEventTypeSegment" date="2020-01-07 11:52:25 -0600" duration="9.412362335125605" durationUnit="min"/>
  <WorkoutEvent type="HKWorkoutEventTypeMotionPaused" date="2020-01-07 11:52:27 -0600"/>
  <WorkoutEvent type="HKWorkoutEventTypeMotionResumed" date="2020-01-07 11:52:28 -0600"/>
  <WorkoutEvent type="HKWorkoutEventTypePause" date="2020-01-07 11:53:03 -0600"/>
  <WorkoutEvent type="HKWorkoutEventTypeResume" date="2020-01-07 11:53:50 -0600"/>
  <WorkoutEvent type="HKWorkoutEventTypeMotionPaused" date="2020-01-07 11:53:51 -0600"/>
  <WorkoutEvent type="HKWorkoutEventTypeMotionResumed" date="2020-01-07 11:53:54 -0600"/>
  <WorkoutEvent type="HKWorkoutEventTypeSegment" date="2020-01-07 11:58:45 -0600" duration="5.411429115136465" durationUnit="min"/>
  <WorkoutEvent type="HKWorkoutEventTypeSegment" date="2020-01-07 12:01:50 -0600" duration="8.241872878869374" durationUnit="min"/>
  <WorkoutEvent type="HKWorkoutEventTypeSegment" date="2020-01-07 12:04:10 -0600" duration="5.078322347005209" durationUnit="min"/>
  <WorkoutEvent type="HKWorkoutEventTypeSegment" date="2020-01-07 12:10:05 -0600" duration="0.2081167499224345" durationUnit="min"/>
  <WorkoutStatistics type="HKQuantityTypeIdentifierActiveEnergyBurned" startDate="2020-01-07 11:52:25 -0600" endDate="2020-01-07 12:10:21 -0600" sum="230.42" unit="Cal"/>
  <WorkoutStatistics type="HKQuantityTypeIdentifierDistanceWalkingRunning" startDate="2020-01-07 11:52:25 -0600" endDate="2020-01-07 12:10:21 -0600" sum="2.01899" unit="mi"/>
  <WorkoutStatistics type="HKQuantityTypeIdentifierBasalEnergyBurned" startDate="2020-01-07 11:52:25 -0600" endDate="2020-01-07 12:10:21 -0600" sum="28.9791" unit="Cal"/>
  <WorkoutRoute sourceName="Jason’s Apple Watch" sourceVersion="13.3" creationDate="2020-01-07 12:34:38 -0600" startDate="2020-01-07 11:52:30 -0600" endDate="2020-01-07 12:10:19 -0600">
   <MetadataEntry key="HKMetadataKeySyncVersion" value="2"/>
   <MetadataEntry key="HKMetadataKeySyncIdentifier" value="C03A13B7-C4C9-431F-8CD3-A19D8561CEEE"/>
   <FileReference path="/workout-routes/route_2020-01-07_12.10pm.gpx"/>

```

In [63]:
def parse_apple_health_export(xml_file):
    """
    Parses Apple Health's export.xml file to extract workout details.

    Parameters:
    - xml_file (str): Path to the export.xml file.

    Returns:
    - df_workouts (DataFrame): Summary of workouts with top-level details.
    - df_events (DataFrame): Detailed breakdown of workout events.
    """
    def time_difference_minutes(start, end):
        """Helper to calculate time difference in minutes."""
        start_time = datetime.strptime(start, "%Y-%m-%d %H:%M:%S %z")
        end_time = datetime.strptime(end, "%Y-%m-%d %H:%M:%S %z")
        return (end_time - start_time).total_seconds() / 60
    
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Lists to hold parsed data
    workouts = []
    #events_list = []
    # Parse workout data
    for i, workout in enumerate(root.findall('Workout')):
        # Extract top-level workout attributes
        workout_data = {
            'ActivityType': workout.get('workoutActivityType', 'N/A').replace("HKWorkoutActivityType", ""),
            'Duration (min)': float(workout.get('duration', 0)),
            'Source': workout.get('sourceName', 'N/A'),
            'Source_version': workout.get('sourceVersion', 'N/A'),
            'StartDate': workout.get('startDate', 'N/A'),
            'EndDate': workout.get('endDate', 'N/A')
        }
        
        # Extract Metadata Entries
        for metadata in workout.findall('MetadataEntry'):
            key = metadata.get('key', '')
            value = metadata.get('value', '')
            if key == "HKAverageMETs":
                workout_data['AverageMETs'] = float(value.replace(" kcal/hr·kg",""))
            if key == "HKIndoorWorkout":
                workout_data['IndoorWorkout'] = value

        # Loop through WorkoutStatistics and dynamically extract attributess
        for stat in workout.findall('WorkoutStatistics'):
            stat_type = stat.get('type', '').replace("HKQuantityTypeIdentifier", "")
            unit = stat.get('unit', '')
            # Dynamically check for sum, average, minimum, and maximum
            for attr in ['sum', 'average', 'minimum', 'maximum']:
                value = stat.get(attr)
                if value:  # Only add if the attribute exists
                    column_name = f"{stat_type}_{attr}_{unit}".strip()
                    workout_data[column_name] = float(value)
        
        # Initialize tracking pause durations variables
        paused_duration = 0.0
        motion_paused_time = None
        pause_time = None
        
        # Flatten WorkoutEvent data and calculate paused durations
        for event in workout.findall('WorkoutEvent'):
            event_type = event.get('type', 'N/A')
            event_date = event.get('date', 'N/A')            

            # Handle paused/resumed events
            if event_type == "HKWorkoutEventTypeMotionPaused":
                motion_paused_time = event_date
            elif event_type == "HKWorkoutEventTypeMotionResumed" and motion_paused_time:
                paused_duration += time_difference_minutes(motion_paused_time, event_date)
                motion_paused_time = None

            if event_type == "HKWorkoutEventTypePause":
                pause_time = event_date
            elif event_type == "HKWorkoutEventTypeResume" and pause_time:
                paused_duration += time_difference_minutes(pause_time, event_date)
                pause_time = None

        # Add calculated paused duration to workout summary
        workout_data['PausedDuration_mins'] = round(paused_duration, 2)
        workouts.append(workout_data)

    # Convert to DataFrames
    df_workouts = pd.DataFrame(workouts)
    
    return df_workouts

In [64]:
my_workouts = parse_apple_health_export(xml_file)

In [65]:
my_workouts.head()

,ActivityType,Duration (min),Source,Source_version,StartDate,EndDate,IndoorWorkout,ActiveEnergyBurned_sum_Cal,BasalEnergyBurned_sum_Cal,PausedDuration_mins,...,RunningVerticalOscillation_maximum_cm,RunningSpeed_average_mi/hr,RunningSpeed_minimum_mi/hr,RunningSpeed_maximum_mi/hr,RunningStrideLength_average_m,RunningStrideLength_minimum_m,RunningStrideLength_maximum_m,HeartRate_average_count/min,HeartRate_minimum_count/min,HeartRate_maximum_count/min
0,HighIntensityIntervalTraining,54.242769,Jason’s Apple Watch,5.3.3,2019-12-12 12:20:45 -0600,2019-12-12 13:15:00 -0600,0,461.313000,95.110000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HighIntensityIntervalTraining,52.620327,Jason’s Apple Watch,5.3.3,2019-12-16 11:49:35 -0600,2019-12-16 12:42:12 -0600,0,628.034000,92.232000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TraditionalStrengthTraining,50.564955,Jason’s Apple Watch,5.3.3,2019-12-17 11:52:57 -0600,2019-12-17 12:45:38 -0600,0,363.550000,88.375800,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cycling,46.048452,Jason’s Apple Watch,5.3.3,2019-12-19 11:57:07 -0600,2019-12-19 12:43:10 -0600,1,569.988000,80.711700,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cycling,10.352728,Jason’s Apple Watch,6.1.1,2019-12-26 17:08:01 -0600,2019-12-26 17:18:23 -0600,0,0.041244,0.172624,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
my_workouts.tail()

,ActivityType,Duration (min),Source,Source_version,StartDate,EndDate,IndoorWorkout,ActiveEnergyBurned_sum_Cal,BasalEnergyBurned_sum_Cal,PausedDuration_mins,...,RunningVerticalOscillation_maximum_cm,RunningSpeed_average_mi/hr,RunningSpeed_minimum_mi/hr,RunningSpeed_maximum_mi/hr,RunningStrideLength_average_m,RunningStrideLength_minimum_m,RunningStrideLength_maximum_m,HeartRate_average_count/min,HeartRate_minimum_count/min,HeartRate_maximum_count/min
694,Walking,5.090908,Jason’s Apple Watch,11.1,2024-11-16 18:59:56 -0600,2024-11-16 19:05:02 -0600,0,7.38105,8.78876,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.3207,76.0,102.0
695,Walking,22.325235,Jason’s Apple Watch,11.1,2024-11-19 12:09:33 -0600,2024-11-19 12:31:53 -0600,0,75.19350,38.55570,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.7440,98.0,105.0
696,Running,7.030009,Jason’s Apple Watch,11.1,2024-11-20 16:18:28 -0600,2024-11-20 16:40:45 -0600,0,79.36400,12.57710,15.3,...,10.5,5.98898,3.24817,7.14057,0.986667,0.8,1.06,139.0000,134.0,143.0
697,TraditionalStrengthTraining,43.713250,Jason’s Apple Watch,11.1,2024-12-11 15:37:08 -0600,2024-12-11 16:20:51 -0600,0,288.94200,73.29250,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.5540,83.0,153.0
698,Walking,19.507575,Jason’s Apple Watch,11.2,2024-12-14 11:04:50 -0600,2024-12-14 11:24:20 -0600,0,59.36130,33.14070,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.1179,90.0,104.0


In [67]:
my_workouts.describe()

,Duration (min),ActiveEnergyBurned_sum_Cal,BasalEnergyBurned_sum_Cal,PausedDuration_mins,AverageMETs,DistanceCycling_sum_mi,DistanceWalkingRunning_sum_mi,DistanceSwimming_sum_yd,SwimmingStrokeCount_sum_count,StepCount_sum_count,...,RunningVerticalOscillation_maximum_cm,RunningSpeed_average_mi/hr,RunningSpeed_minimum_mi/hr,RunningSpeed_maximum_mi/hr,RunningStrideLength_average_m,RunningStrideLength_minimum_m,RunningStrideLength_maximum_m,HeartRate_average_count/min,HeartRate_minimum_count/min,HeartRate_maximum_count/min
count,699.000000,697.000000,665.000000,699.000000,657.000000,4.000000e+00,311.000000,6.000000,6.000000,21.000000,...,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,83.000000,83.000000,83.000000
mean,33.868744,295.723560,54.525250,0.548941,7.982494,1.029251e-01,2.400590,833.333333,326.166667,3983.209524,...,10.823810,6.400204,4.534100,7.500130,1.031756,0.868571,1.166667,125.533888,96.096386,149.831325
std,54.603344,138.356434,23.076810,2.650898,3.171258,1.131527e-01,0.841849,258.198890,96.636260,1165.268094,...,0.254764,0.264544,1.020154,0.488318,0.037542,0.064520,0.076376,30.371957,30.098057,28.515216
min,0.000000,0.041244,0.129883,0.000000,1.300000,2.792940e-10,0.012913,500.000000,189.000000,1011.000000,...,10.300000,5.988980,2.378210,6.762470,0.979882,0.750000,1.060000,78.201600,58.000000,89.000000
25%,23.463122,220.405000,39.745600,0.000000,4.872380,4.254510e-02,2.022160,625.000000,257.500000,3620.000000,...,10.600000,6.217330,3.732630,7.140570,0.999251,0.830000,1.120000,101.872000,76.000000,129.000000
50%,29.875018,308.143000,49.828400,0.000000,9.640530,7.443960e-02,2.636780,1000.000000,385.000000,4589.800000,...,10.800000,6.414350,4.689940,7.433130,1.037390,0.870000,1.160000,115.301000,84.000000,154.000000
75%,42.039918,351.186000,71.951000,0.020000,10.762400,1.348196e-01,3.010450,1000.000000,385.750000,4733.000000,...,11.000000,6.553160,5.300500,7.671370,1.055700,0.910000,1.200000,158.315000,106.000000,176.000000
max,1432.133333,651.749000,208.195000,53.480000,12.843400,2.628210e-01,4.406260,1000.000000,397.000000,4972.000000,...,11.400000,7.105500,6.041670,8.947080,1.128460,0.980000,1.330000,184.194000,176.000000,195.000000


In [69]:
# Filter to just running workouts
my_workouts[my_workouts['ActivityType'] =='Running'].tail(5)

,ActivityType,Duration (min),Source,Source_version,StartDate,EndDate,IndoorWorkout,ActiveEnergyBurned_sum_Cal,BasalEnergyBurned_sum_Cal,PausedDuration_mins,...,RunningVerticalOscillation_maximum_cm,RunningSpeed_average_mi/hr,RunningSpeed_minimum_mi/hr,RunningSpeed_maximum_mi/hr,RunningStrideLength_average_m,RunningStrideLength_minimum_m,RunningStrideLength_maximum_m,HeartRate_average_count/min,HeartRate_minimum_count/min,HeartRate_maximum_count/min
672,Running,28.275990,Jason’s Apple Watch,10.6.1,2024-10-02 10:24:16 -0600,2024-10-02 10:52:53 -0600,0,351.969,49.7976,0.33,...,10.8,6.43732,4.17089,7.14054,1.041560,0.80,1.14,174.235,157.0,186.0
675,Running,27.999573,Jason’s Apple Watch,10.6.1,2024-10-08 10:01:00 -0600,2024-10-08 10:29:00 -0600,0,350.316,48.6920,0.00,...,11.0,6.51519,2.37821,7.44314,1.056300,0.95,1.20,168.330,131.0,181.0
687,Running,29.980431,Jason’s Apple Watch,11.1,2024-10-31 13:21:04 -0600,2024-10-31 13:51:12 -0600,0,358.556,52.2307,0.15,...,11.4,6.15706,2.99211,7.43070,0.988563,0.87,1.08,180.906,165.0,187.0
692,Running,29.495730,Jason’s Apple Watch,11.1,2024-11-14 13:44:47 -0600,2024-11-14 14:14:27 -0600,0,350.271,51.2176,0.17,...,11.0,6.21733,3.66544,6.76247,0.999251,0.75,1.07,177.271,169.0,186.0
696,Running,7.030009,Jason’s Apple Watch,11.1,2024-11-20 16:18:28 -0600,2024-11-20 16:40:45 -0600,0,79.364,12.5771,15.30,...,10.5,5.98898,3.24817,7.14057,0.986667,0.80,1.06,139.000,134.0,143.0


In [70]:
# Save the outputs
my_workouts.to_csv("output/workout_summary.csv", index=False)

print("Main Workout Summary:")
print(my_workouts.head())


Main Workout Summary:
                    ActivityType  Duration (min)               Source  \
0  HighIntensityIntervalTraining       54.242769  Jason’s Apple Watch   
1  HighIntensityIntervalTraining       52.620327  Jason’s Apple Watch   
2    TraditionalStrengthTraining       50.564955  Jason’s Apple Watch   
3                        Cycling       46.048452  Jason’s Apple Watch   
4                        Cycling       10.352728  Jason’s Apple Watch   

  Source_version                  StartDate                    EndDate  \
0          5.3.3  2019-12-12 12:20:45 -0600  2019-12-12 13:15:00 -0600   
1          5.3.3  2019-12-16 11:49:35 -0600  2019-12-16 12:42:12 -0600   
2          5.3.3  2019-12-17 11:52:57 -0600  2019-12-17 12:45:38 -0600   
3          5.3.3  2019-12-19 11:57:07 -0600  2019-12-19 12:43:10 -0600   
4          6.1.1  2019-12-26 17:08:01 -0600  2019-12-26 17:18:23 -0600   

  IndoorWorkout  ActiveEnergyBurned_sum_Cal  BasalEnergyBurned_sum_Cal  \
0             0     